# Topic Modeling for 20-Newsgroups

There are several approaches to topic modeling. The most popular options are Latent Dirichlet Allocation (LDA) and Non-negative Matrix Factorization (NMF). In this notebook we will use the 20-newsgroups dataset to compare these methods with probabilistic Latent Semantic Analysis (pLSA) and ensemble topic modeling (EnsTop) from the enstop library. This is not meant to be a particularly complete or comprehensive comparison, but rather a means to show how the enstop library works, and provide a quick comparison to other popular approaches.

First we'll need the requisite libraries. Fortunately sklearn has a function to get the 20-newsgroups dataset, a CountVectorizer which can convert the raw text data into bag-of-words based count matrix, and implementations of both LDA and NMF. We'll of course also need the PLSA and EnsembleTopics classes from the enstop library.

In [1]:
import warnings; warnings.simplefilter('ignore') # Suppress deprecation warnings

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from enstop import EnsembleTopics, PLSA

/Users/leland/anaconda3/envs/enstop_dev/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/leland/anaconda3/envs/enstop_dev/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


The next step is getting the data. For this we can just use sklearn. First the ``fetch_20newsgroups`` function will download the 20-newsgroups data. By specifying ``subset='all'`` we collect the full dataset rather than either a train or test set. The next step in the process is to convert this text data into a form that can be consumed by LDA, NMF, PLSA and EnsembleTopics. The required format, in this case, is a matrix where the (i,j)th entry is the count of the number of times the jth word in the vocabulary occurs in the ith document (in this case each document is a newsgroup post). This can be done extremely efficiently using sklearn's ``CountVectorizer``. We'll pass two extra parameters to the ``CountVectorizer``: a setting of ``min_df=5`` which will restrict the vocabulary to words that occur at least 5 times in the entire corpus; and ``stop_words='english'`` which will eliminate common words (like "the", "and", etc.) accordingly to a dictionary of such words in English.

In [3]:
news = fetch_20newsgroups(subset='all')
data = CountVectorizer(min_df=5, stop_words='english').fit_transform(news.data)

## Time to fit a model

Now that we have the data is order, let's fit the various topic models and time them to see how long they take to fit. First up is LDA. The only parameter that requires tuning in this case is the number of topics we want to have. As a reasonable guess we'll choose 20 (the number of different newsgroups in the dataset).

In [4]:
%%time
lda_model = LatentDirichletAllocation(n_components=20).fit(data)

CPU times: user 4min 5s, sys: 2.61 s, total: 4min 8s
Wall time: 2min 10s


A little over two minutes on my laptop -- not bad at all. It is worth noting the total CPU time, which came in at a little over four minutes, demonstrating that the implementation is making good use of parallelism, especially considering this is running on a two core processor.

Next up is NMF. In this case we need a few extra parameters for the sklearn implementation. By default the sklearn NMF uses Frobenius loss -- essentially the total squared error between the data matrix and the reconstruction from the product two low rank matrices (with positive entries). While this is suitable for many uses it isn't the right loss for topic modeling. Instead we want to use the Kullback-Leibler loss, which essentially models the data as a set of independent Poisson's -- essentially it views the data as counts (which they are), and seeks the reconstruction from the product two low rank matrices to provide Poisson parameters that maximise the likelihood of seeing the data. Having changed the loss function we also need to change the solver from the classical coordinate descent to the multiplicative update based solver which can work with KL loss. All of this makes the NMF fitting process much slower, but it provides more accurate results for the purposes of topic modelling.

In [5]:
%%time
nmf_model = NMF(n_components=20, beta_loss='kullback-leibler', solver='mu').fit(data)

CPU times: user 3min 46s, sys: 1min 26s, total: 5min 12s
Wall time: 3min 25s


Around three and half minutes, so slower than LDA in this case. In general, especially for larger datasets than this, NMF will often tend to be as fast or sometimes even faster than LDA. In this case, however, it is a little slower. It's again worth noting the CPU time: over five minutes. Again, the implementation is making good use of parallelism on the two core processor.

Next let's try EnsembleTopics. In this case we will specify ``n_components=20`` as with LDA and NMF, but this time that is more of a suggestion. EnsembleTopics will attempt to find a "natural" number of topics. Given that this is a small dataset we will also reduce the overall work to be done via the ``n_starts`` parameter, which specifies how many bootstrap runs of pLSA to try; for small data like this eight runs will likely suffice rather than the default 15. It is also beneficial to scale the parallelism a little -- since the processor only has two cores it is best not to overtax it with too many jobs at once.

In [6]:
%%time
ens_model = EnsembleTopics(n_components=20, n_starts=8, n_jobs=2).fit(data)

CPU times: user 9min 42s, sys: 3.31 s, total: 9min 45s
Wall time: 5min 4s


Coming in at over five minutes this is by far the slowest of the options we've tried here -- more than twice as long as LDA. On the other hand, as with NMF, EnsembleTopics will scale up well, and would likely look better on larger datasets. It is, however, in the right ballpark, which is somewhat reassuring. When we look at how well the topic modeling performed on the data this extra time might seem more worthwhile.

Lastly let's look at pLSA. Historically pLSA is a precursor to LDA which came out a couple of years later and added Bayesian priors and more robust statistical foundations. On the other hand the pLSA algorithm itself is surprisingly simple, and with a little care high performance implementations are not hard to write. Given an efficient Expectation-Maximization optimizer it can potentially even find better solutions than a somewhat more complex LDA optimization. Using pLSA from enstop is just as easy as LDA in sklearn -- tell it the number of topics you want and set it going.

In [7]:
%%time
plsa_model = PLSA(n_components=20).fit(data)

CPU times: user 39.6 s, sys: 336 ms, total: 39.9 s
Wall time: 40 s


A mere forty seconds! Decidedly faster than LDA, and it will scale well in terms of dataset size (but may scale less well in the desired number of topics). At the very least, given its speed, pLSA is a contender in the topic modeling space. Also worth noting is that this performance was achieved despite being a completely serial implementation -- the CPU time is the same as the wall time in this case.

Now, having looked at how long it takes the algorithms to run, the next question is: how good are they? A fast algorithm that does a poor job is not worth much.

## Quality measures of topic models

In [8]:
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score
import numpy as np

In [9]:
nmf_doc_vectors = nmf_model.transform(data)
nmf_clusters = np.argmax(nmf_doc_vectors, axis=1)
lda_doc_vectors = lda_model.transform(data)
lda_clusters = np.argmax(lda_doc_vectors, axis=1)
ens_doc_vectors = ens_model.embedding_
ens_clusters = np.argmax(ens_doc_vectors, axis=1)
plsa_doc_vectors = plsa_model.embedding_
plsa_clusters = np.argmax(plsa_doc_vectors, axis=1)

In [10]:
print("NMF Adjusted Rand: ", adjusted_rand_score(news.target, nmf_clusters))
print("NMF Adjusted Mutual Information: ", adjusted_mutual_info_score(news.target, nmf_clusters))

NMF Adjusted Rand:  0.1524803617829113
NMF Adjusted Mutual Information:  0.3234024294426364


In [11]:
print("LDA Adjusted Rand: ", adjusted_rand_score(news.target, lda_clusters))
print("LDA Adjusted Mutual Information: ", adjusted_mutual_info_score(news.target, lda_clusters))

LDA Adjusted Rand:  0.20401323297635027
LDA Adjusted Mutual Information:  0.3406073183799349


In [12]:
print("pLSA Adjusted Rand: ", adjusted_rand_score(news.target, plsa_clusters))
print("pLSA Adjusted Mutual Information: ", adjusted_mutual_info_score(news.target, plsa_clusters))

pLSA Adjusted Rand:  0.2885965535771851
pLSA Adjusted Mutual Information:  0.44955493998742224


In [13]:
print("EnsTop Adjusted Rand: ", adjusted_rand_score(news.target, ens_clusters))
print("EnsTop Adjusted Mutual Information: ", adjusted_mutual_info_score(news.target, ens_clusters))

EnsTop Adjusted Rand:  0.3591142637691707
EnsTop Adjusted Mutual Information:  0.5154491050775457
